In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import scipy as sp
import numpy as np
import emcee
import testkit
import corner
import pickle as pickle
from IPython.display import display
%matplotlib inline
import forwardmodel
import ciamod
import TPmod
import cloud
import band
import brewtools
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian1DKernel
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from bensconv import spex_non_uniform
from bensconv import conv_uniform_R
from bensconv import conv_uniform_FWHM

## First step is to load the results file and the run arguments.
We open the files and select the last couple of thousand iterations of the chain, and flatted this into a simple array of state vectors called "samples"

In [ ]:
#path = "/you/can/add/a/path/to/your/results/files"
runname = "<yourrunname>"
# OK finish? 1 for yes, 0 for no.
fin = 0
flatendchain, flatendprobs,ndim = brewtools.get_endchain(runname,fin)
theta_max_end = flatendchain[np.argmax(flatendprobs)]
max_end_like = np.amax(flatendprobs)
samples = flatendchain

argfile =runname+"_runargs.pic"

runargs = brewtools.pickle_load(argfile)
# If you're opening on a Linux box, use the code below
#with open(argfile, 'rb') as input:
#    runargs = pickle.load(input) 

gases_myP,chemeq,dist, cloudtype,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype,do_fudge, prof,do_bff,bff_raw,ceTgrid,metscale,coscale = runargs



Then we print the max likelihood value for the state vector, just to have a look at it


In [ ]:
print(theta_max_end)

And print the BIC for this run. You might want to take a note of this.


In [ ]:
BIC = (-2.* max_end_like) + (ndim*np.log(obspec.shape[1]))
print("BIC = "+str(BIC))

# Next we want to plot the profile. 

We start by calculating the profiles from the temperature parameters in the samples array.

This case assumes a type 2 profile.


In [ ]:
Tsamples = samples[:,ndim-5:]
nsamps = Tsamples.shape[0]
Tprofs = np.empty([64,Tsamples.shape[0]])
for i in range(0,nsamps):
    Tprofs[:,i] = TPmod.set_prof(2,coarsePress,press,Tsamples[i,:])
    
Tlays = np.empty([64,5])
for i in range(0,64):
    junk = Tprofs[i,:]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    Tlays[i,:] = junk3[:]
    

Now we can plot it...

In [ ]:
plt.rc('font',family='Times New Roman')
fig=plt.figure(dpi=320)
plt.axis([0., 4000.,3.0,-5.0])

logP = np.log10(press)

d1, = plt.plot(Tlays[:,2],logP,'k-',label=runname)
plt.fill_betweenx(logP,Tlays[:,1], Tlays[:,3], facecolor='red', alpha=0.3)
plt.fill_betweenx(logP,Tlays[:,0], Tlays[:,4], facecolor='red', alpha=0.1)



# Here are some condensation curves

enst = 10.0**4/(6.26 - 0.35*logP-0.70*0.0)
fost = 10.0**4/(5.89 - 0.37*logP-0.73*0.0)
iron = 10.0**4/(5.44 - 0.48*logP-0.48*0.0)
cr =  10.0**4/(6.528 - 0.491*logP-0.491*0.0)
al2o3 = 10.0**4 / (5.0139 - 0.21794*(logP) + 2.2636E-03*(logP)**2.0 - 0.580*0.0)
c1, = plt.plot(enst,logP,'--',color='blue',linewidth=1.5, label='MgSiO$_3$')
c2, = plt.plot(fost,logP,'--',color='pink',linewidth=1.5,label='Mg$_2$SiO$_4$')
c3, = plt.plot(iron,logP,'--',color='orange',linewidth=1.5, label='Fe')
c4, = plt.plot(cr,logP,'--',color='purple',linewidth=1.5, label='Cr')
c5, = plt.plot(al2o3,logP,'--',color='red',linewidth=1.5, label='Al$_2$O$_3$')

plt.legend(handles=[d1,c4,c1,c2,c3,c5])
plt.ylabel(r'log(P / bar)')
plt.xlabel('T / K')

#plt.savefig(runname+"_profile.png",format='png', dpi=320)

In [ ]:
# Save the median profile
np.savetxt(runname+'_profile.dat', np.c_[logP, Tlays[:,2]])

Now we plot the gravity and composition parameters in a corner plot

In [ ]:
gassamples = samples[:,0:3]
rmet = (-1.0, +2.0)
rco = (0, 2)
rlogg = (3.5,5.5)
bnds = [rmet,rco,rlogg]

fig = corner.corner(gassamples,scale_hist=False, range = bnds,plot_datapoints =False,labels=["[Fe/H]","[C/O]","logg"],quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize": 20},label_kwargs={"fontsize": 20})

#plt.savefig(runname+"_gascorner.png",format='png', dpi=320)

# Plotting the model spectra
To plot the model spectra we need to rerun the model, and rerun it for a bunch of random draws from the posterior. This will allow us to plot either a spaghetti plot, or a median + interval spectrum

In [ ]:
# get diagnostics along with the spectrum
gnostics = 0
# Now run the model again to get your model spectrum and process to make it look like the data
shiftspec, photspec, tauspec,cfunc = testkit.modelspec(theta_max_end,runargs,gnostics)
topspec = brewtools.proc_spec(shiftspec,theta_max_end,fwhm,chemeq,gasnum,obspec) 

In [ ]:
# Now grab 500 random draws from the posterior
pltspec = np.zeros((500,obspec[0,:].size))
samp= np.empty(ndim)
samples = flatendchain
sid = np.zeros(500)
for i in range (0,500):
    sid[i]= np.random.randint(0,high = len(samples))
    samp = samples[int(sid[i]),:]
    shiftspec, photspec, tauspec,cfunc = testkit.modelspec(samp,runargs,gnostics)
    pltspec[i,:] = brewtools.proc_spec(shiftspec,samp,fwhm,chemeq,gasnum,obspec) 

# get the intervals for the distribution of model spectra
specdist = np.empty([obspec[0].size,5])
for i in range(0,obspec[0].size):
    junk = pltspec[:,i]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    specdist[i,:] = junk3[:]

In [ ]:
# plot the spectra
plt.rc('font',family='Times New Roman')
fig=plt.figure(dpi=320)
plt.axis([1.0,2.5,0,1e-14])
# you can swap in the lines below for a log plot
#plt.axis([0.6,15.0, 1.e-17,150e-15])

#ax = fig.add_subplot(1,1,1)

#ax.set_yscale('log')
#ax.set_xscale('log')

# add in below lines for spaghetti plot
#for i in range(0,500):
#    plt.plot(obspec[0],pltspec[i,:],color='r',alpha=0.01)

t1, = plt.plot(obspec[0,:],topspec,'g-',linewidth=1, label = "max likelihood")
d1, = plt.plot(obspec[0,:],obspec[1,:],'k-',label = runname+" data")
#plt.fill_between(obspec[0,:],obspec[1,:]-obspec[2,:],obspec[1,:]+obspec[2,:],facecolor='red',alpha=0.2)

r1, = plt.plot(obspec[0],specdist[:,2],'y-',linewidth=0.5, label = "median")
plt.fill_between(obspec[0],specdist[:,0],specdist[:,4],facecolor='red',alpha=0.2)
plt.fill_between(obspec[0],specdist[:,1],specdist[:,3],facecolor='red',alpha=0.5)


plt.legend(handles=[d1,t1,r1])


plt.ylabel(r'$ F_{\lambda}$ / $Wm^{-2} \mu m^{-1}$')
plt.xlabel('Wavelength / $\mu m$')
#plt.savefig(runname+"_SPAG_SPEC.png",format='png', dpi=320)

In [ ]:
# Save you  model spectra
np.savetxt(runname+'_MEDIAN_SPEC.dat', np.c_[obspec[0,:],specdist[:,2],specdist[:,1],specdist[:,3]])
np.savetxt(runname+'_MAX_LIKE_SPEC.dat', np.c_[obspec[0,:],topspec])np.savetxt(runname+'_MEDIAN_SPEC.dat', np.c_[obspec[0,:],specdist[:,2],specdist[:,1],specdist[:,3]])

# Contribution function
Finally, the contribution function. This is a really useful plot...

In [ ]:
# get diagnostics along with the spectrum
gnostics = 1
shiftspec, clphotspec, ophotspec,cfunc = testkit.modelspec(theta_max_end,runargs,gnostics)

nwave = inwavenum.size
cfunc = np.reshape(cfunc,[cfunc.shape[1],cfunc.shape[2]])
fwhm = 0.005
wlen = shiftspec.shape[1]
wint =  shiftspec[0,0] - shiftspec[0,wlen-1]
# convolve with instrumental profile
# start by setting up kernel
# First step is finding the array index length of the FWHM
disp = wint / wlen
gwidth = int((((fwhm / disp) // 2) * 2) +1)
# needs to be odd
# now get the kernel and convolve
gauss = Gaussian1DKernel(gwidth)

for ilayer in range (0,press.size):
    cfunc[:,ilayer] = convolve(cfunc[:,ilayer],gauss,boundary='extend')

tau1_cl_Press = convolve(clphotspec[0],gauss,boundary='extend')[::-1]
tau1_oth_Press = convolve(ophotspec[0],gauss,boundary='extend')[::-1]
    
wavenew = shiftspec[0,::-1]
press = press.reshape(64,)
normfunc = np.zeros_like(cfunc)
for iwave in range(0,nwave):
    totcont = np.sum(cfunc[iwave,:])
    normfunc[iwave,:] = cfunc[iwave,:] / totcont

    
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
fig=plt.figure(dpi=120)
plt.axis([1.0,15.0,1000,1e-5])

ax = fig.add_subplot(1,1,1)

ax.set_yscale('log')
ax.set_xscale('log')
#major_ticks = np.arange(1.0,15.,1.0)
#minor_ticks = np.arange(1.0,15.,0.5)
#ax.set_xticks(major_ticks)                                                       
#ax.set_xticks(minor_ticks, minor=True)                                           

plt.pcolormesh(wavenew,press,(normfunc[::-1,:].transpose()),cmap='Greys',norm=colors.SymLogNorm(linthresh=0.001,linscale=0.00001,
                                              vmin=0., vmax=np.amax(normfunc)))

#t1, = plt.plot(wavenew,(tau1_cl_Press),'m-',label=r'$\tau_{cloud} = 1.0$')
t2, = plt.plot(wavenew,(tau1_oth_Press),'c-', label =r'$\tau_{gas} = 1.0$')

plt.legend(handles=[t2])

cbar = plt.colorbar(orientation='vertical',norm=colors.Normalize(clip=False),ticks=[1e-3,1e-2,0.1])
cbar.ax.set_yticklabels(['<0.1%', '1%', '10%'])
cbar.set_label('% of total', rotation=270)
plt.ylabel('Pressure / bar)')
plt.xlabel('Wavelength / $\mu m$')
#plt.savefig(runname+'_contribution.png',format='png', dpi=120)